In [34]:
# neural network with variable input, hidden and output nodes
# implemets the sigmoid activation function
# uses the backpropagation algorithm

In [35]:
import numpy

In [36]:
class neuralNetwork:
    
    def __init__(self, inputnodes, hiddennodes, outputnodes, categories):
        # number of nodes in each input, hidden, output layer
        self.inodes = inputnodes
        self.hnodes = hiddennodes
        self.onodes = outputnodes
        
        # random initial weights arrays wif and who
        # structure is w_i_j where i is current layer, j is next layer
        # w11 w21
        # w12 w22 etc 
        self.wih = numpy.asarray((numpy.random.rand(self.hnodes, self.inodes) - 0.5), dtype='complex128')
        self.wih += ( 1j * (numpy.random.rand(self.hnodes, self.inodes) - 0.5))
        self.wih /= self.inodes
        self.who = numpy.asarray((numpy.random.rand(self.onodes, self.hnodes) - 0.5), dtype='complex128')
        self.who += ( 1j * (numpy.random.rand(self.onodes, self.hnodes) - 0.5))
        self.who /= self.hnodes
        
        # cast to higher range dtype
        self.wih = numpy.asarray(self.wih, dtype='complex128')
        self.who = numpy.asarray(self.who, dtype='complex128')
        
        # number of categories
        self.cats = categories
        pass
    
    def z_to_class(self, z):
        # first work out the angle, but shift angle from [-pi, +pi] to [0,2pi]
        angle = numpy.mod(numpy.angle(z) + 2*numpy.pi, 2*numpy.pi)
        # from angle to category
        p = numpy.floor (self.cats * angle / (2*numpy.pi))
        return p

    def class_to_angle(self, p):
        # class to angle, using bisector
        angle = ((p + 0.5) / self.cats) * 2 * numpy.pi
        return angle
    
    def status(self):
        print ("wih = \n", self.wih)
        print ("who = \n", self.who)
        print ("cats = ", self.cats)
        pass

    def query(self, inputs_list):
        # convert inputs list to 2d array
        inputs = numpy.array(inputs_list, ndmin=2, dtype='complex128').T
        
        # calculate hidden layer outputs
        hiddeninputs = numpy.dot(self.wih, inputs)
        hiddenoutputs = numpy.exp(1j * numpy.angle(hiddeninputs))
        
        # calculate output layer outputs
        finalinputs = numpy.dot(self.who, hiddenoutputs)
        finaloutputs = numpy.exp(1j * numpy.angle(finalinputs))
        
        return self.z_to_class(finaloutputs)
    
    def train(self, inputs_list, targets_list):
        # convert inputs list to 2d array
        inputs = numpy.array(inputs_list, ndmin=2, dtype='complex128').T
        targets = numpy.array(targets_list, ndmin=2, dtype='complex128').T
        
        # calculate hidden layer outputs
        hiddeninputs = numpy.dot(self.wih, inputs)
        hiddenoutputs = numpy.exp(1j * numpy.angle(hiddeninputs))
        
        # calculate output layer outputs
        finalinputs = numpy.dot(self.who, hiddenoutputs)
        finaloutputs = numpy.exp(1j * numpy.angle(finalinputs))
        
        # error at output layer
        desired_angles = self.class_to_angle(targets)
        output_errors =  numpy.exp(1j*desired_angles) - finaloutputs
        output_errors /= self.hnodes
        
        self.who += numpy.dot(output_errors, numpy.conj(hiddenoutputs.T))
        
        # hidden layer error is the output_errors, split by weights, recombined at hidden nodes
        hidden_errors = numpy.dot(self.who.T, output_errors)
        hidden_errors /= (self.inodes * numpy.abs(hiddeninputs))
        
        self.wih += numpy.dot(hidden_errors, numpy.conj(inputs.T))
        
        #print("wih after = \n", self.wih)
        #print("who after = \n", self.who)
    pass

In [37]:
# number of input, hidden and output nodes
# chosen for mnist character data
inodes = 784
hnodes = 200
onodes = 1

# categories = 10
categories = 10

# create instance of neural network
n = neuralNetwork(inodes, hnodes, onodes, categories)

In [38]:
# read mnist training data
f = open("mnist_train.csv", 'r')
mnist_train_csv = f.readlines()
f.close()

In [39]:
#train neuralnetwork

# epochs is number of times entire training set is used for learning
epochs = 5

for e in range(epochs):
    for line in mnist_train_csv:
        linebits = line.split(',')
        targets = numpy.asarray(int(linebits[0]))
        # put 0-255 range to -1.0 to +1.0
        inputs = (numpy.asfarray(linebits[1:]) / 255.0 * 0.99) + 0.01
        inputs = numpy.exp(1j * 1 * numpy.pi * inputs)
        n.train(inputs, targets)
        pass
    pass

In [40]:
testindex = 20
line = mnist_train_csv[testindex]
linebits = line.split(',')
target = linebits[0]
print(target)
inputs = (numpy.asfarray(linebits[1:]) / 255.0 * 0.99) + 0.01
inputs = numpy.exp(1j * 1 * numpy.pi * inputs)
n.query(inputs)

4


array([[ 5.]])

In [41]:
# read mnist test data
f = open("mnist_test.csv", 'r')
mnist_test_csv = f.readlines()
f.close()

In [42]:
# list to keep record of correct or incorrect predictions
record = []

# work through each test example
for line in mnist_test_csv:
    linebits = line.split(',')
    #print(linebits[0])
    answer = int(linebits[0])
    # put 0-256 range to -1.0 to +1.0
    inputs = (numpy.asfarray(linebits[1:]) / 255.0 * 0.99) + 0.01
    inputs = numpy.exp(1j * 1 * numpy.pi * inputs)
    # finaloutputs is the answer from the already trained neuralnetwork
    out_class = n.query(inputs)
    # append correct or incorrect to list
    if (answer == int(out_class)):
        record.append(1)
    else:
        record.append(0)
        pass
    pass

In [43]:
print(record[:100])
print(len(record))

[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0]
10000


In [44]:
r = numpy.asarray(record)
print (r.sum() / r.size)

0.3292
